<a href="https://colab.research.google.com/github/Mitr-yuzr/Mirai-NLP-Notebook/blob/main/mirai_nlp_gpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mirai-NLP-GPT2

___

> 基于[gpt2-chinese模型](https://github.com/Morizeyao/GPT2-Chinese)的文本生成语言模型训练笔记本

作者 **Mitr-yuzr**  
GITHUB: [Mitr-yuzr](https://github.com/Mitr-yuzr)  
QQ: 1758919843  
EMAIL: craft.yuzr@qq.com  
MIRAI FORUM: [@MITR-YUZR](https://mirai.mamoe.net/user/mitr-yuzr)


## 声明

___

仅供学习用途，禁止用于任何违反法律法规和社区规定的行为，禁止用于商业行为。

## 使用方法

___

- 收集并导出 `gpt2` 格式数据
- 登录 Google 账号并确保 Google Drive 可用
- 点击右上角的**连接**，等待分配完成和初始化
- 在上方菜单栏中选择**代码执行程序**，在倒数第三项找到**更改运行时类型**，在**硬件加速器**中选择 `GPU`，等待分配完成
- 在下方按照提示依序进行初始化、训练和生成

In [ ]:
#@title 全局变量 { run: "auto" }

#@markdown ### 模型名字
model_name = "mirai-gpt2" #@param {type:"string"}

## 初始化

以下单元格为训练和生成内容前的初始化准备，必须从上往下运行。

若为二次训练，则**无需进行 ``下载模型`` 步骤**。

In [ ]:
#@title 挂载Google Drive
#@markdown 训练好的模型都会保存在 ``Google Drive`` 中以方便下载、使用和分享，请自行确保``Google Drive``可以正常使用。

#@markdown 若连接时没有自动挂载谷歌硬盘，可运行此单元格进行挂载。

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title 下载源码
#@markdown 可能会需要比较长的时间，等待完成即可。
!git clone https://github.com/Morizeyao/GPT2-Chinese
%cd GPT2-Chinese
!pip install -r requirements.txt
!gdown 1iga4-oFO-ST55VnLnVRFmHEavmN_DLeb

In [ ]:
#@title 下载预训练模型 { display-mode: "form" }

#@markdown 这里使用的是[hhou435](https://github.com/hhou435)大佬提供的通用中文小模型。

#@markdown 你可以前往[此处](https://github.com/Morizeyao/GPT2-Chinese#%E6%A8%A1%E5%9E%8B%E5%88%86%E4%BA%AB)下载其他模型 (散文、诗词、对联、歌词、文言文等)，并自行上传到``drive/MyDrive/your_model_name/``文件夹下。

#@markdown 若为二次训练 (即已经在GDrive中有训练过的模型)，则无需运行此单元格。

%cd /content/drive/MyDrive/
!mkdir {model_name}
!mkdir {model_name}/pretrain_model
%cd /content/drive/MyDrive/{model_name}/pretrain_model

!gdown 1J_PV3m2OGseTbIyCfbejY28xaShHGevr
!gdown 1MXAXWEDGNRH0wEc-qRFbTGWWTZS7EZ2G
!gdown 1g18-G0okJM8NRVy6iDySvNRDigQGysz1

%cd /content/GPT2-Chinese

In [ ]:
#@title 上传数据
#@markdown 点击运行，在**输出日志**中会出现上传文件的入口，选择刚刚导出的**gpt2**的数据并上传 (会自动重命名为``train.json``)。

#@markdown 请检查确认为gpt2类型 (内容只有一行列表，每项只有内容) 的数据后再上传，否则会出现严重问题。

!mkdir '/content/GPT2-Chinese/data'
%cd data

import os
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  os.rename(fn, "train.json")
%cd ..

## 训练

本单元格为训练单元格，请仔细填写好参数后运行本单元格。

___

In [ ]:
#@title  { display-mode: "form" }

#@markdown ### 模型地址
#@markdown 若第一次训练则为 ``pretrain_model``，若二次训练则为上次训练最后的模型的文件夹名 (如: ``model_epoch10``)。
pretrain_model_name = "pretrain_model" #@param {type:"string"}

#@markdown ### 切片数量
#@markdown 对训练数据进行切片的数量，推荐为 ``数据量 / batch_size / 10``。
pieces_num = 5 #@param {type:"integer"}

#@markdown ### 批大小
#@markdown 每批的数据量，依每条数据的长度进行调整。推荐为 ``2、4、8、16``。

#@markdown 合适的批大小可以使训练效果更好，但此项参数过大则会引发内存溢出 (Out of Memory, OOM) 错误。

#@markdown 若出现OOM错误，则可以将此参数调小。
batch_size = 8 #@param {type:"integer"}

#@markdown ### 训练轮数
#@markdown 将所有训练数据都训练一遍的次数，依据总数据量进行调整。

#@markdown 轮数过少会导致**欠拟合**，轮数过多会导致**过拟合**，合适的轮数可以使训练效果更好。

#@markdown 推荐训练一定轮数后生成文本查看效果，若过拟合则使用先前模型，若欠拟合则在当前模型的基础上进行二次训练。
epochs = 10 #@param {type:"integer"}

#@markdown ### 模型保存间隔
#@markdown 每隔多少轮保存一次模型，推荐小一点，以防止过拟合或colab背刺。
save_every = 5 #@param {type:"integer"}

#@markdown ### 起始轮数
#@markdown 若首次训练则填0，若二次训练则填下使用的模型的已训练论数。

#@markdown 仅用于文件名方便，不影响训练效果。
start_epoch = 0 #@param {type:"integer"}

!python ./train.py \
  --pretrained_model /content/drive/MyDrive/{model_name}/{pretrain_model_name} \
  --model_config /content/drive/MyDrive/{model_name}/{pretrain_model_name}/config.json \
  --tokenizer_path /content/drive/MyDrive/{model_name}/{pretrain_model_name}/vocab.txt \
  --save_every {save_every} \
  --num_pieces {pieces_num} \
  --raw \
  --epochs {epochs} \
  --batch_size {batch_size} \
  --output_dir /content/drive/MyDrive/{model_name}/ \
  --start_epoch {start_epoch}

## 生成文本

In [ ]:
#@title 生成参数 { run: "auto", display-mode: "form" }
#@markdown ___

#@markdown ### 模型名字
model = "model_epoch10" #@param {type:"string"}

#@markdown ### 生成内容开头
prefix = "\u6211\u662F" #@param {type:"string"}

#@markdown ### 生成长度
length = 20 #@param {type:"integer"}

#@markdown ### 生成的数量
nsamples = 5 #@param {type:"integer"}

In [ ]:
#@title 直接生成 { display-mode: "form" }
!python ./generate.py \
  --length={length} \
  --nsamples={nsamples} \
  --prefix={prefix} \
  --fast_pattern \
  --model_path /content/drive/MyDrive/{model_name}/{model} \
  --model_config /content/drive/MyDrive/{model_name}/{model}/config.json \
  --tokenizer_path /content/drive/MyDrive/{model_name}/pretrain_model/vocab.txt

In [ ]:
#@title 生成并保存至本地 { display-mode: "form" }
!python ./generate.py \
  --length={length} \
  --nsamples={nsamples} \
  --prefix={prefix} \
  --fast_pattern \
  --model_path /content/drive/MyDrive/{model_name}/{model} \
  --model_config /content/drive/MyDrive/{model_name}/{model}/config.json \
  --tokenizer_path /content/drive/MyDrive/{model_name}/pretrain_model/vocab.txt \
  --save_samples

from google.colab import files
files.download('/content/GPT2-Chinese/samples.txt')